In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.environ["DATA_DIR"])

/data_store


In [25]:
from omegaconf import OmegaConf, DictConfig
from pathlib import Path
import os
os.environ['CONFIG_DIR'] = os.getcwd() + "/configs"

base_config = OmegaConf.load(Path(os.environ['CONFIG_DIR']) / "base_config.yml")

def flatten_config(config, parent_key=None, sep='_'):
    items = []
    for k, v in config.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, DictConfig):
            items.extend(flatten_config(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return DictConfig(dict(items))

flattened_config = flatten_config(base_config)

flattened_config

{'experiment_project': 'ViTSSM', 'experiment_group': 'ViT', 'experiment_name': 'vmdsprites', 'experiment_log_freq': 25, 'dataset_name': 'vmdsprites', 'dataset_mode': 'train', 'dataset_fold': '0', 'dataset_resolution': 64, 'dataset_train_percentage': 0.9, 'dataset_num_workers': 8, 'dataset_pin_memory': True, 'optimization_loss_name': 'MSELoss', 'optimization_optimizer_name': 'AdamW', 'optimization_optimizer_kwargs_lr': 0.001, 'optimization_optimizer_kwargs_weight_decay': 0.01, 'optimization_scheduler_name': 'StepLR', 'optimization_scheduler_kwargs_step_size': 1, 'optimization_scheduler_kwargs_gamma': 0.5, 'optimization_epochs': 10, 'optimization_batch_size': 32, 'optimization_val_batch_size': 32, 'metrics': None, 'model_device': 'cuda', 'model_use_amp': True}

In [7]:
from vitssm.data.datasets import VideoMDSpritesDataset


dataset = VideoMDSpritesDataset("data_store/VMDsprites")

In [3]:
from vitssm.data import get_dataloaders
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("run_configs") / "base_config.yml")
config = OmegaConf.load(Path("run_configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

dataloaders = get_dataloaders(conf)

In [4]:
from tqdm import tqdm

for _ in tqdm(dataloaders[0]):
    pass

100%|██████████| 226/226 [00:19<00:00, 11.48it/s]


In [1]:
from vitssm.data.datasets import VideoMDSpritesDataset, NextFrameDataset
from vitssm.models.next_frame_prediction import LatentNextFramePrediction

dataset = NextFrameDataset(VideoMDSpritesDataset("data_store/VMDsprites", download=True))

/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE


In [12]:
from torchvision.transforms.v2 import Compose, ToDtype
import torch

model = LatentNextFramePrediction(
    frame_in_size=(64, 64),
    frame_out_size=(64, 64),
    n_frames=99,
    patch_size=4,
    latent_dim=64,
    n_heads=2,
)

x = ToDtype(torch.float32, scale=True)(dataset[0][0].unsqueeze(0))
print(x.shape)
pred = model(x)
print(pred.shape)

torch.Size([1, 99, 64, 64, 3])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([1, 99, 64, 64, 3])


In [8]:
from torchvision.transforms.v2 import Compose, ToDtype
import torch


tensor(1.)

In [3]:
import wandb

wandb.init(config="/home/david/projects/ViTSSM/run_configs/base_config.yml")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


TypeError: string indices must be integers, not 'str'

In [8]:
x, y = dataset[0]

In [3]:
import gdown

file_id = '1T7Vq7a70hu5949FJMe8yie5tvAwOmxXV'
url = f'https://drive.google.com/uc?id={file_id}'
# Specify output location
output = 'VMDsprites.zip'  # Replace with desired file name

# Download the file
gdown.download(url, output, quiet=False)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1T7Vq7a70hu5949FJMe8yie5tvAwOmxXV
From (redirected): https://drive.google.com/uc?id=1T7Vq7a70hu5949FJMe8yie5tvAwOmxXV&confirm=t&uuid=6a458c62-33fa-459d-a0c2-96a42732cbae
To: /home/david/projects/ViTSSM/VMDsprites.zip
100%|██████████| 272M/272M [00:07<00:00, 38.1MB/s] 


'VMDsprites.zip'

In [2]:
from vitssm.data.mdsprites.generators import generate_videos
from vitssm.data.mdsprites.shapes import Circle, Rectangle, Triangle

gen = iter(generate_videos(
    64,
    [Circle, Rectangle, Triangle],
    6,
    ["red", "blue", "green", "yellow", "purple"],
    "#000000",
    100,
))

for _ in range(10):
    print(next(gen)[1])

[[array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255,

In [6]:
from xformers.components.attention import ScaledDotProduct
from xformers.ops.fmha.attn_bias import LowerTriangularMask
import torch
from torch import nn

att = ScaledDotProduct(seq_len=30, to_seq_len=40)
q_proj = nn.Linear(128, 128)
k_proj = nn.Linear(128, 128)
v_proj = nn.Linear(128, 128)

q = torch.ones((32, 30, 128))
k = torch.ones((32, 30, 128))
v = torch.ones((32, 40, 128))
mask = LowerTriangularMask()

att(q_proj(q), k_proj(k), v_proj(v), mask=mask).shape

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [32, 30] but got: [32, 40].

In [1]:
from vitssm.models.next_frame_prediction import LatentNextFramePrediction
import torch

model = LatentNextFramePrediction(frame_in_size=(32, 32), frame_out_size=(32, 32), patch_size=4, latent_dim=32, n_heads=2, n_frames=10)
x = torch.randn(32, 10, 32, 32, 3)
out = model(x)
print(out.shape)

/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE
/home/david/micromamba/envs/ViTSSM/lib/python3

torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([32, 10, 32, 32, 3])


In [3]:
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("run_configs") / "base_config.yml")
config = OmegaConf.load(Path("run_configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

In [1]:
from torchvision.datasets import MovingMNIST
from vitssm.data.datasets import NextFrameDataset


mnist = MovingMNIST(root="data", split="train", split_ratio=10, download=False)
data = NextFrameDataset(mnist)


(tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         ...,
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
      